In [ ]:
# First we import all the necessary packages
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# We will follow the EDA workflow explained in lecture 13a

# 1. Load & Initial Reconnaissance
raw_data = pd.read_csv("data/data.csv")

# After loading the data we perform an initial exploration to understand the structure and content of the data set
print(
    f"An initial exploration of the data shows that there are {raw_data.shape[0]} enties (rows) that represent each car, and {raw_data.shape[1]} attributes (columns) that determine each car feature. All columns are object, float64 and int64 data types. However, some columns could be converted to categorical variables to make a better use of the data and optimize performace."
)
print(
    "The columns that could potentially be converted to categorical variables are: Make, Model, Year, Engine Fuel Type, Transmission Type, Driven_Wheels, Number of Doors, Market Category, Vehicle Size, and Vehicle Style."
)
print(
    "The only column that represents time data is the year. However, it is no woth converting it to a datetime variable since the analysis can be done considering it a categorical variable."
)
print(
    f"\nWe can also see that the non-null count for some of the variables(columns) is different than the total number of rows ({raw_data.shape[0]}), which indicates that there are missing values. In particular, the columns missing some values are: Engine Fuel Type, Engine HP, Engine Cylinders, Number of Doors, and Market Category.\n"
)
display(raw_data.info())


# rows print
print(
    "\nBy looking at the first and last rows of the data we can deduce several things. Firstly, we notice that the data seems to be ordered by the Model column, since it starts with those models whose first character is a '1' and ends with those whose first character is 'z'. It could also be ordered according to the Popularity rating, since the first cars have a high number while  the last one has a small one."
)
print(
    "\nThe information on most of the columns is obvious, like: Make, Model, Year, Engine HP, Engine Cylinders, Transmission Type, Driven_Wheels, Number of Doors, Vehicle Size, Vehicle Style, highway MPG, city mpg and MSRP (Manufacturer's Suggested Retail Price). However, the columns Market Category and Popularity require some explanation. The Market Categorty column stores different words describing the car, like: Luxury, Performance, Factory Tuner, etc. It will be necessary to transform the data on this column to be able to leverage the information contained in it. The Popularity column stores the popularity rating for every car. It might need to be normalized to better understand and represent the data."
)
print("\nFirst five rows:")
display(raw_data.head())
print("\nLast five rows:")
display(raw_data.tail())
print("\nNow we explore the number of categories for each variable:\n")
print(
    raw_data[
        [
            "Make",
            "Model",
            "Year",
            "Engine Fuel Type",
            "Transmission Type",
            "Driven_Wheels",
            "Number of Doors",
            "Market Category",
            "Vehicle Size",
            "Vehicle Style",
        ]
    ].nunique()
)


# Transmission Type
print(
    "\n\nWe are surpreised to find that there are five different transmission types, since we only expected to find two (manual and automatic). Below are all the transmission types.\n"
)
print(raw_data["Transmission Type"].value_counts())
print(
    "\nWe have found that there is category named 'UNKNOWN', which we consider missing data. This indicates that some of the variables could be storing missing values inside of specific categories. In this case the number of missing values is 19."
)


# Driven wheels
print(
    "\n\nAs for the driven wheels we were expecting three different values: front, rear and all wheel drive. Below we print all the alternatives.\n"
)
print(raw_data["Driven_Wheels"].value_counts())
print(
    " \nWe have found out that four wheel drive and all wheel drive are different things, and that explains the fourth category. We will later check all the values for the different variables to verify that there are no missing values stored in any category."
)


# Market Category
print(
    f"The number of possible values for the Market Category variable is very high ({raw_data['Market Category'].nunique()}). The reason for this is that every row combines different possibilities, and that yields a higher number of combinations than the actual number of unique possible values. For example, if ca car A's Market Category says 'Luxury, Performance', and car b's Market Category says 'Luxury' each row is considered a unique value, although in reality there are two separate attributes/features to differentiate (Luxury and Performance). We will deal with this issue during the Data Cleaning Phase."
)


# Initial statistics
print("\n\nNow we perform some basic statistics on the data:")
print(raw_data.describe())
print(
    "\nThere aren't many relevant conclusions that we can derive from this analysis. However, there are some interesting inshights. For example, there is a 1000 HP car, a car with 16 cyclinders, popularity ranges between 2 and 5657 with an average of 1554.9 (this discards the idea that the data was originally ordered according to the Popularity rating), a car with a highway milage of 354 MPG, and the cars years ranges between 1990 and 2007. This potential outliers will be investigated in the Data Cleaning Phase.\n"
)


print(
    "After an initial exploration of the data we conclude that this data set provides information about different car models and their characteristics, including: MSRP, mileage, HP, etc.The data can be considered tidy, except for the Market Category column, There are also some missing values and potential outliers that need to be investigated."
)
print(
    "Our objective is to understand the dataset and learn from it. Thats why we ask ourselves the following questions which we will try to answer in the EDA phase."
)
print(
    "- What numerical characteristics are related to one another. To do this we will do a correlation heatmap?"
)
print(
    "- How many cars have been produced each year. Does it follow a trend? Is there any year with more cars?"
)
print("- Is there a relationship between the type of transmission of the car and its horsepower?")
print("- What brands and models are more popular and why?")
print("- What is the relationship between HP and MPG?")
print(
    "- What is the relationship between price and the category of the car (luxury, performance, etc.)?"
)
print("- What other factors affect price")

In [ ]:
# 2. Data Quality Assessment
print(
    "In this phase we will assess the quality of the data. To do so we will explore the values of the different categories, looking for variable categories storing missing values. We will also look at the missing values and outliers that have already been identified in the data."
)

# Exploration of the categories and missing values

# Make
# print(raw_data['Make'].unique()) #I comment this line of code for cleanliness in the output, but it was used to explore the different values of the Make variable
# Model
# print(raw_data['Model'].unique()) #I comment this line of code for cleanliness in the output, but it was used to explore the different values of the Model variable


# Market category
# print(raw_data['Market Category'].unique()) #I comment this line of code for cleanliness in the output, but it was used to explore the different values of the Market Category variable
print(
    "\n\nAs outlined in the previous section, the number of possible values for the Market Category variable is very high because every row combines different possibilities, and that yields a higher number of combinations than the actual number of unique possible values."
)
Market_Categories = raw_data["Market Category"].str.split(",").explode().str.strip().unique()
print(
    f"After filtering all the values, we conclude that the Market Category variable can take any combination of {len(Market_Categories)} unique possible values:"
)
print(Market_Categories)
print(
    "As expected, since the initial analysis revealed that there are {raw_data['Market Category'].isna().sum()} missing valus in the Market Category column, there is a category called 'nan' storing missing values. We will give the missing values the name 'NO CATEGORY' in the next phase.\n"
)

print(
    "\nNow, we will analyze the other categories in order to find which are the possible values they can take and ensure no missing values are passed to the analysis phase.\n"
)

# Fuel type
print(raw_data["Engine Fuel Type"].value_counts())
print()
# Transmission Type
print(raw_data["Transmission Type"].value_counts())
print()
# Driven_Wheels
print(raw_data["Driven_Wheels"].value_counts())
print()
# Number of Doors
print(raw_data["Number of Doors"].value_counts())
print()
# Vehicle Size
print(raw_data["Vehicle Size"].value_counts())
print()
# Vehicle Style
print(raw_data["Vehicle Style"].value_counts())
print()

print(
    "Thanks to this analysis we have found that Transmission type contains a category for missing values called 'UNKONWN' with a total count of 19. We suspected that UNKNOWN category for Transmission Type correspond to electric vehicles, and the values could be reassigned to a new specific category. However, that is not the case. All the vehicles in that category use gas, and thus the values will simply be changed to NaN values."
)
# raw_data[raw_data['Transmission Type']=='UNKNOWN'] #I comment this line of code for cleanliness in the output, but it was used for the analysis of the Transmission Type category
raw_data["Transmission Type"] = raw_data["Transmission Type"].replace(
    "UNKNOWN", np.nan
)  # to include UNKNOWN value as missing data
print(
    "After adding that variable value to the list of NaN values, the total count of missing values is:\n"
)
print(raw_data.isna().sum()[lambda x: x > 0])
print(
    "\n\nWe have found that the quality of the data is good, with sufficient information for an analysis, a good organization and structure. However, there are some some missing values and potential outliers (like 1000 Engine HP or 354 highway MPG) that need to be handled. In addition, the types need to be switched to categorical for most of the variables."
)
print(
    f"Moreover, the values in the Market Category variable contain a combination multiple classifications. By extracting and splitting those pieces of information a better understanding of the data could be achieved, and the subsequent analysis would be simplified. To better understand this take row 0 as an example: row 0 stores in the Market Category the value: {raw_data.loc[0, 'Market Category']} which could be split into 3 categories."
)
print(
    "In conclusion, the data requires to be cleaned and processed to make it adequate for the subsequent analysis."
)

In [ ]:
# 3. Cleaning Decisions
print(
    "In this phase we will take decisions and actions regarding the cleaning and processing of the data."
)
# Firstly, we will reassing the variable types to the best suitable type.
print("We begin by reassigning the variable type to all the columns that require it.\n")
raw_data = raw_data.astype(
    {
        "Make": "category",
        "Model": "category",
        "Engine Fuel Type": "category",
        "Transmission Type": "category",
        "Driven_Wheels": "category",
        "Number of Doors": "category",
        "Vehicle Size": "category",
        "Vehicle Style": "category",
    }
)

# we confirm that the variable type has been converted.
print(raw_data.dtypes)

# Now we explore the missing values
print(
    "\nNext, we explore the missing values in the data. We can see that there are missing values on six different categories, but the count of missing values is very different. While Engine Fuel Type is only missing 3 values, market category is missing 3742.\n"
)
print(raw_data.isna().sum()[lambda x: x > 0])

print(
    "\n\nNext, we analyze the rows with missing data, in search of any patterns or reasons for that data to be missing, and with the intention to fix the missing values when possible:\n"
)

# IGNORE the following commented lines
# print("Engine Fuel Type:")
# print(raw_data[raw_data['Engine Fuel Type'].isna()])
# print("\nAll the rows missing the value of the Engine Fuel Type correspond to Suzuki Veronas. If we filter all the rows of Suzuki Verona we see that their Engine Fuel Type is regular unleaded in all cases. However, all values missing are from 2004 and it would be risky to assume that prior models were all regular unleaded as well. Therefore, we will simply drop teh missing values\n")
# print(raw_data[raw_data['Model']=='Verona'])

# print("\n\nNumber of doors:")
# print(raw_data[raw_data['Number of Doors'].isna()])
# print("\nMost of the rows missing the value of the Number of Doors correspond to Tesla Model S cars. Since all Tesla Model S cars have 4 doors we will manually correct the missing data replacing it with a 4 ")
# print(raw_data[raw_data['Model']=='Model S'])


# Engine Fuel Type & Number of Doors
print(
    f"Dropping the rows missing the values for engine fuel type and number of doors results in losing 9 rows, which we consider acceptable compared to the size of the data set ({raw_data.shape[0]}) ."
)


# Engine HP
print(
    "\n\nWe suspect that most the rows missing values for Engine HP might be electric, and that most electric vehicles don't have a value for Engine HP in most of the cases."
)
missing_values_hp = raw_data[(raw_data["Engine HP"].isna())]
print(
    f"\nRows missing Engine HP data with sorted by Engine Fuel Type: {missing_values_hp['Engine Fuel Type'].value_counts()[lambda x: x > 0]}"
)
print(
    f"\nTotal amount of rows with 'Electric' as Engine Fuel Type: {(raw_data['Engine Fuel Type'] == 'electric').sum()}"
)
print(
    "This indicates that 44 out of the 66 electric vehicle entries don't include the Engine HP. We will try to correct the missing values, when possible."
)
print(
    "Looking at the raw data of the rows with 'electric' as Engine Fuel Type and missing Engine HP, we can conclude that see that Tesla Model S and Nissan Leaf take most of the missing values."
)
print(missing_values_hp.groupby("Model", observed=True).size())
print(
    f"\nIn particular Tesla Model S accounts for {missing_values_hp[missing_values_hp['Model'] == 'Model S'].shape[0]} rows of missing data and Nissan leaf for {missing_values_hp[missing_values_hp['Model'] == 'Leaf'].shape[0]} rows of missing data."
)
# print(raw_data[(raw_data['Engine Fuel Type']=='electric') & (raw_data['Engine HP'].isna() == True)]) #I comment this part of code to simplify output, but it was used to investigae rows with missing data about Engine HP
# print(raw_data[raw_data['Model']=='Model S'][['Make','Model','Engine HP']]) #I comment this out, but it was used to verify that there is no data about Tesla Model S Enginge HP
# print(raw_data[raw_data['Model']=='Leaf'][['Make','Model','Engine HP']]) #I comment this out, but it was used to verify that there is no data about Nissan Leaf Enginge HP
print(
    "Unfortunately, non of the data enties for Tesla Model S or Nissan Leaf include information about the HP that we could assume equal to all the models, so we will have to drop all rows missing values for Engine HP."
)


# Enigne Cylinders
missing_values_cylinders = raw_data[raw_data["Engine Cylinders"].isna()]
print(
    f"\n\nNow we look at the rows missing data about the Engine Cylinders, and find that Mazda RX-8 accounts for {missing_values_cylinders[missing_values_cylinders['Model'] == 'RX-8'].shape[0]} of the {missing_values_cylinders.shape[0]} total missing."
)
# print(missing_values_cylinders[['Make','Model','Engine Cylinders']]) #I comment this part of the code to simplify the output, bu it was used to investigate rows missing data about Engine Cylinders
print(missing_values_cylinders.groupby("Model", observed=True).size())
# print(raw_data[raw_data['Model']=='RX-8'][['Make','Model','Engine Cylinders']]) #I comment this out, but it was used to verify that there is no data about Mazda RX-8 Enginge Cylinders
print(
    "Unfortunately, once again there is no information about this model on the other rows that could be used to replace the NaN values, therefore all rows with missing values for Engince Cylinders will be dropped."
)


# Transmission Type
missing_values_transmission = raw_data[raw_data["Transmission Type"].isna()]
print(
    f"\n\nNow we look at the rows missing data about the Transmission Type, and find that Dodge RAM 150 accounts for {missing_values_transmission[missing_values_transmission['Model'] == 'RAM 150'].shape[0]} of the {missing_values_transmission.shape[0]} total missing."
)
print(missing_values_transmission.groupby("Model", observed=True).size())
# print(missing_values_transmission[['Make','Model','Transmission Type']]) #I comment this part of the code to simplify the output, bu it was used to investigate rows missing data about Transmission Type
# print(raw_data[raw_data['Model']=='RAM 150']) #I comment this out, but it was used to verify that the rows of Dodge RAM 150 could be used to fill missing values on other rows
print(
    "\nIn this case we can see that there are more entries for this car model and they all have similar data. Therefore we will assume that the transmission was manual on the rows with missing information for the Dodge RAM 150."
)
raw_data["Transmission Type"] = raw_data["Transmission Type"].fillna("MANUAL")
# print(raw_data[raw_data['Model']=='RAM 150'][['Make','Model','Transmission Type']]) #verify that the change was made


# Market category
print(
    f"\n\nSince the missing values in the Market Category column are considerable ({raw_data['Market Category'].isna().sum()}) and the different unique possible values of this variable are too many ({len(Market_Categories)}) a new category will be created and assigned to those missing values."
)
raw_data["Market Category"] = raw_data["Market Category"].fillna("NO CATEGORY")
print(
    "At this point, we've decided to split the values on every row for the Market Category column into a list of strings. This will help with the subsequent analysis, facilitating access to the data. This decision required us to go iterate and not transform the Market Category into a categorical variable to be able to store the values in a list. The new category for missing values is now a list with a singel value 'NO CATEGORY'"
)
raw_data["Market Category"] = (
    raw_data["Market Category"].str.split(",").apply(lambda lst: [x.strip() for x in lst])
)


# Clean data from missing values
print(
    "\n\nIn conclusion, to clean the data we have dediced to drop the missing values for all the variables except for those missing the Transmission Type corresponding to Dodge RAM 150, and those for the Market Category, which will be reassigned to a new category. We've demonstated that we can handle missing values by dropping them or replacing them when adequate. Additionaly, the values of the Market Category column are now stored in a list on every row instead of a string."
)
clean_data = raw_data.dropna(
    subset=[
        "Engine Fuel Type",
        "Engine HP",
        "Engine Cylinders",
        "Transmission Type",
        "Number of Doors",
    ]
)


# Outliers
print("\n\nWe also want to explore some of the potential outliers:")

# Potential outlier 16 Engine Cylinders
print(
    f"At the beginning of our analysis we saw that there was a car with 16 Engine Cylinders, which sounds like too many. After inspecting the rows corresponding to that value to validate the data entry we have found that the car model is a {raw_data[raw_data['Engine Cylinders'] == 16]['Make'].unique()[0]} {raw_data[raw_data['Engine Cylinders'] == 16]['Model'].unique()[0]},  which is a sport car and explains the size of the engine and the HP. Therefore, it is considered legitimate and not an outlier."
)
# print(raw_data[raw_data["Engine Cylinders"] == 16]) #I comment this row for cleanliiness in the output, but it was used to analyze a potential outlier.


# Potential outlier highway MPG 354
print(
    f"\nWe also found an entry indicating a vehicle with a highway mileage of 354MPG, which doesn't seem to be very realistic. After inspecting that row to validate the data entry we have found that the car model is an {raw_data[raw_data['highway MPG'] == 354]['Make'].unique()[0]} {raw_data[raw_data['highway MPG'] == 354]['Model'].unique()[0]}. After comparing this value with those of similar models we have decided to drop this row."
)
# print(raw_data[raw_data["highway MPG"] == 354]) #I comment this line of code, but it was used to verify that there was an outlier
print(raw_data[raw_data["Model"] == "A6"][["Make", "Model", "highway MPG"]])
clean_data = clean_data.drop(clean_data[clean_data["highway MPG"] == 354].index)


# Potential outlier city MPG 137
print(
    f"\nThere is also an entry of a car with a city mileage of 137 mpg, which doesn't seem to be very realistic. After inspecting that row to validate the data entry we have found that the car model is an {raw_data[raw_data['city mpg'] == 137]['Make'].unique()[0]} {raw_data[raw_data['city mpg'] == 137]['Model'].unique()[0]}."
)
# print(raw_data[raw_data["city mpg"] == 137]) #I comment this line of code, but it was used to verify that there was an outlier
# print(clean_data[clean_data['Engine Fuel Type'] =='electric']) #I comment this line of code, but it was used to verify that there was an outlier
print(
    "In this case the vehicle is electric/hybrid and the city mpg most likely refers to the combined range. Therefore, it is considered legitimate and not an outlier."
)


# Eliminate duplicated values
clean_data_duplicates = clean_data.copy()
# To delete the duplicated rows the values in the Market Category column had to be converted into tuples.
clean_data_duplicates["Market Category"] = clean_data_duplicates["Market Category"].apply(
    lambda x: tuple(x) if isinstance(x, list) else x
)
clean_data = clean_data_duplicates.copy().drop_duplicates()
print(
    f"As a final step of the cleaning process all the duplicated rows are eliminated. There is a total of {clean_data_duplicates.duplicated().sum()} duplicated rows that are being deleted."
)


# End of Phase 2
print(
    "\nWe are aware that there might be more outliers, or wrong data to be cleaned. However, at this point it is not possible to further screen the data. A visual analysis of the data is required to continue with the data exploration."
)

In [ ]:
# Save the cleaned data in a pickle file
clean_data.to_pickle(
    "C:/Users/gongi/miniconda3/envs/insy6500/my_repo/projects/final_project/data/cleaned_data.pkl"
)